
# End-To-End ETL Financial Pipline 
## Utilizing Python, APIs, Web Scraping and SQL to consolidate and process financial data from multiple types of sources

In [1]:
 # Install Necessary resources 
!pip install nltk
!pip install TextBlob
!pip install Beautifulsoup4
!pip install googletrans==4.0.0-rc1
!pip install nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 636.8/636.8 kB 2.2 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 817.4 kB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 6.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 9.5 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17398 sha256=775ade5b71953515278b3c6846c322656e88c218a789233035b895234ead4c0d
  Stored in directory: /Users/kymair/Library/Caches/pip/wheels/c0/59/9f/7372f0cf70160fe61b528532e1a7c8498c4becd6bcffb022de
Succe

In [4]:
#Import Needed Libraries and Necessary resources from NLTK, Beautiful Soup, TextBlob, Google Translate
#import html5lib
from datetime import datetime
from bs4 import BeautifulSoup
from textblob import TextBlob
import numpy as np
import pandas as pd
import requests
import io
import nltk
from nltk import word_tokenize
from googletrans import Translator 
import os
import sqlite3 as sql


# 1) Extraction

In [5]:
#1a) Fetch Financial News headlines
cnbc_URL = "https://www.cnbc.com/stocks/"
cnbc_headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'}
cnbc_r = requests.get(cnbc_URL, headers=cnbc_headers)
cnbc_timestamp=datetime.now()



In [6]:
#1b)Fetch Stock Market Data 
aapl_url='https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=AAPL&interval=60min&outputsize=compact&apikey=LJ7H7F9D9Z7ECUK7&datatype=csv'
r_aapl=requests.get(aapl_url)
msft_url='https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=MSFT&interval=60min&outputsize=compact&apikey=LJ7H7F9D9Z7ECUK7&datatype=csv'
r_msft=requests.get(msft_url)
goog_url='https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=GOOG&interval=60min&outputsize=compact&apikey=LJ7H7F9D9Z7ECUK7&datatype=csv'
r_goog=requests.get(goog_url)
amzn_url='https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=AMZN&interval=60min&outputsize=compact&apikey=LJ7H7F9D9Z7ECUK7&datatype=csv'
r_amzn=requests.get(amzn_url)
meta_url='https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=META&interval=60min&outputsize=compact&apikey=LJ7H7F9D9Z7ECUK7&datatype=csv'
r_meta=requests.get(meta_url)

# 2) Transformation

## Financial News Headline Data 

In [7]:
# Using the extracted Financial News headline data, retrieve the url for the headline
html_text = cnbc_r.text
soup = BeautifulSoup(html_text, 'lxml')
card_titles=soup.find_all('a', class_='Card-title')
headlines_data=[]
for title in card_titles:
    headlines_data.append(title['href'])


In [8]:
# Extract title from url and strip .html then create a list of sentences for each url 
df_holder=pd.DataFrame(headlines_data, columns=['url'])
df_holder[['http','empty', 'site', 'year','month','date','title']] = df_holder.url.str.split("/",expand=True)
titles=df_holder['title'].str.replace('-', ' ').tolist()
sentences =  [x[:-4] for x in titles]



In [9]:
#Create a data frame for the headlines and add the columns for title, url, timestamp
df_headlines=pd.DataFrame(sentences, columns=['headline'])
df_headlines['url']=df_holder['url']
df_headlines['timestamp'] = cnbc_timestamp

In [10]:
#2a. Add the sentiment score and sentiment for each headline sentence using the toekenize functions
df_headlines['sentiment_score']=df_headlines['headline'].apply(lambda sent: TextBlob(sent).sentiment.polarity)
def f(score):
    if -1 <= score  and score < -0.2: return "negative"
    elif -0.2 <= score and score > 0.2: return "neutral" 
    else: return "positive"
assign_sentiment = lambda x: list(map(f, x))
df_headlines['sentiment']=assign_sentiment(df_headlines['sentiment_score'])
      
    

In [11]:
#2b. Tokenize the text into individual "relevant words" by NLTK
df_headlines['relevant_words']=df_headlines['headline'].apply(word_tokenize) 

LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt/PY3/english.pickle[0m

  Searched in:
    - '/Users/kymair/nltk_data'
    - '/Users/kymair/anaconda3/envs/analytics/nltk_data'
    - '/Users/kymair/anaconda3/envs/analytics/share/nltk_data'
    - '/Users/kymair/anaconda3/envs/analytics/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - ''
**********************************************************************


In [10]:
#2c. Create new translated column
my_translator = Translator()
df_headlines['headline_es']=df_headlines['headline'].apply(lambda x: my_translator.translate(x, src='en', dest='es').text)
df_headlines['headline_it']=df_headlines['headline'].apply(lambda x: my_translator.translate(x, src='en', dest='it').text)


In [12]:
#View Data
df_headlines.index.name = "ID"
df_headlines.head(5)

,headline,url,timestamp,sentiment_score,sentiment
ID,,,,,
0,china economy july industrial output fixed ass...,https://www.cnbc.com/2023/08/15/china-economy-...,2023-08-14 21:13:22.117892,0.100000,positive
1,forget china top alternatives stocks to buy ac...,https://www.cnbc.com/2023/08/15/forget-china-t...,2023-08-14 21:13:22.117892,0.500000,neutral
2,with ai stocks down jim cramer says many not b...,https://www.cnbc.com/2023/08/14/with-ai-stocks...,2023-08-14 21:13:22.117892,0.214815,neutral
3,cramers lightning round stay away from plug po...,https://www.cnbc.com/2023/08/14/cramers-lightn...,2023-08-14 21:13:22.117892,-0.200000,positive
4,to own great stock you must prepare for the lo...,https://www.cnbc.com/2023/08/14/to-own-great-s...,2023-08-14 21:13:22.117892,0.700000,neutral


In [13]:
# Create file folders (Made it manually but included the code to make it within the code)
os.makedirs("data")
os.makedirs("data/headlines")

In [14]:
# 2d. Save the headlines DataFrame to a CSV file 
df_headlines.index = df_headlines.index.rename('ID')
df_headlines.to_csv('data/headlines/headlines_data.csv', encoding='utf-8')

## Stocks Data Transformation

In [15]:
#Read the stock market data 
aapl_content=r_aapl.content
aapl = pd.read_csv(io.StringIO(aapl_content.decode('utf-8')))
msft_content=r_msft.content
msft = pd.read_csv(io.StringIO(msft_content.decode('utf-8')))
goog_content=r_goog.content
goog = pd.read_csv(io.StringIO(goog_content.decode('utf-8')))
amzn_content=r_amzn.content
amzn = pd.read_csv(io.StringIO(amzn_content.decode('utf-8')))
meta_content=r_meta.content
meta = pd.read_csv(io.StringIO(meta_content.decode('utf-8')))

In [16]:
# In prepraration of merge each stock dataframe into one, Lets add the stock name as a column named symbol 
aapl['symbol']='AAPL'
msft['symbol']='MSFT'
goog['symbol']='GOOG'
amzn['symbol']='AMZN'
meta['symbol']='META'

In [17]:
#Merge the stocks into one data frame and rename the columns to match the assignment
df_stocks=aapl
df_stocks.append(msft, ignore_index = True)
df_stocks.append(goog, ignore_index = True)
df_stocks.append(amzn, ignore_index = True)
df_stocks.append(meta, ignore_index = True)
dict={"timestamp":"date", "open":"open_price", "high":"highest_price", "low":"lowest_price", "close":"close_price"}
df_stocks.rename(columns=dict,inplace=True)

/var/folders/wn/mnlxj0jn0d771znx5hvhf3g40000gn/T/ipykernel_63456/3370357267.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_stocks.append(msft, ignore_index = True)
/var/folders/wn/mnlxj0jn0d771znx5hvhf3g40000gn/T/ipykernel_63456/3370357267.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_stocks.append(goog, ignore_index = True)
/var/folders/wn/mnlxj0jn0d771znx5hvhf3g40000gn/T/ipykernel_63456/3370357267.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_stocks.append(amzn, ignore_index = True)
/var/folders/wn/mnlxj0jn0d771znx5hvhf3g40000gn/T/ipykernel_63456/3370357267.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  

In [18]:
df_stocks.index = df_stocks.index.rename('ID')

In [20]:
# Save Data
os.makedirs("data/stocks")
df_stocks.to_csv('data/stocks/stocks_data.csv', index=False,encoding='utf-8')
df_stocks.head()

,date,open_price,highest_price,lowest_price,close_price,volume,symbol
ID,,,,,,,
0,2023-08-14 19:00:00,179.800,179.940,179.650,179.940,47601,AAPL
1,2023-08-14 18:00:00,179.500,179.840,179.470,179.810,38389,AAPL
2,2023-08-14 17:00:00,179.460,204.232,162.507,179.525,47045,AAPL
3,2023-08-14 16:00:00,179.515,193.100,177.302,179.430,12804803,AAPL
4,2023-08-14 15:00:00,178.960,179.520,178.830,179.510,6373314,AAPL


# 3) Load

In [21]:
# 3a) Create a db with the schema for the headlines table and the stocks table
db = sql.connect('etl_extended_case.db')
c = db.cursor()


###  Create two tables for the headline and stocks data 

In [22]:
c.execute("""CREATE TABLE headlines(ID INTEGER PRIMARY KEY NOT NULL, 
    headline TEXT, 
    url TEXT,
    timestamp TEXT,
    sentiment_score REAL,
    sentiment TEXT, 
    relevant_words TEXT,
    headline_es TEXT, 
    headline_it TEXT)""")

In [23]:
c.execute("""CREATE TABLE stocks(ID INTEGER PRIMARY KEY NOT NULL, 
    date TEXT, 
    open_price REAL,
    highest_price REAL,
    lowest_price REAL,
    close_price REAL, 
    volume INTEGER,
    symbol TEXT)""")

## Read the data from the CSV files and load into the SQLite database

In [24]:
dfHeadlines = pd.read_csv('data/headlines/headlines_data.csv')
dfHeadlines.to_sql('headlines', db, if_exists='append',index=False)

35

In [25]:
dfStocks = pd.read_csv('data/stocks/stocks_data.csv')
dfStocks.to_sql('stocks', db, if_exists='append',index=False)

100

In [24]:
#check data

In [26]:
c.execute('''SELECT * FROM headlines''')

In [27]:
c.execute('''SELECT * from stocks''')

In [28]:
#check data
c.execute('''SELECT * FROM headlines LIMIT 5''')

In [29]:
c.execute('''SELECT * FROM stocks LIMIT 5''')

In [30]:
db.commit()
db.close()